# 필요한 라이브러리 import

In [40]:
import csv
import pandas as pd
import os
import re

# csv 파일 만드는 함수

In [41]:
def csv_maker(df,path):

    csv_file = open(str(path),'w',newline='',encoding='utf-8')
    csvwriter = csv.writer(csv_file)
    csvwriter.writerow(['이름','WAR'])
    for i in range(len(df)):
        csvwriter.writerow(df.loc[i,['이름','WAR']].tolist())
    csv_file.close()

# war 계산기

In [48]:
def war_calculator(file_path,season_path,forward_weights,defender_weights,midfielder_weights):

    season_re = re.compile('\d+')
    league_re = re.compile('[a-zA-Z0-9]+')
    league_list = league_re.findall(file_path)
    season_list = season_re.findall(season_path)

    all_df = pd.read_csv(file_path,encoding='utf-8')

    forward_save_path = season_path+'/'+'WAR/'+league_list[8]+'-Forward'+'.csv'
    defender_save_path = season_path+'/'+'WAR/'+league_list[8]+'-Defender'+'.csv'
    midfielder_save_path = season_path+'/'+'WAR/'+league_list[8]+'-Midfielder'+'.csv'
    

    forward_df = all_df[all_df['포지션']=='공격수']
    midfielder_df = all_df[all_df['포지션']=='미드필더']
    defender_df = all_df[all_df['포지션']=='수비수']

    forward_var_list = ['이름']+forward_weights.columns.tolist()[0:-1]+['표준획득승점']
    midfielder_var_list = ['이름']+midfielder_weights.columns.tolist()[0:-1]+['표준획득승점']
    defender_var_list = ['이름']+defender_weights.columns.tolist()[0:-1]+['표준획득승점']

    forward_df = forward_df[forward_var_list]
    midfielder_df = midfielder_df[midfielder_var_list]
    defender_df = defender_df[defender_var_list]

    forward_df['WAR'] = 0
    midfielder_df['WAR'] = 0
    defender_df['WAR'] = 0

    forward_df = forward_df.reset_index(drop=True)
    midfielder_df = midfielder_df.reset_index(drop=True)
    defender_df = defender_df.reset_index(drop=True)

    forward_bias = forward_weights.loc[0,'절편']
    midfielder_bias = midfielder_weights.loc[0,'절편']
    defender_bias = defender_weights.loc[0,'절편']


    for i in range(len(forward_df)):

        temp_df = forward_df.iloc[i,:]*forward_weights.iloc[:,0:-1]
        war = temp_df.sum(axis=1).iloc[0]+forward_bias
        forward_df.loc[i,'WAR'] = war

    for i in range(len(midfielder_df)):

        temp_df = midfielder_df.iloc[i,:]*midfielder_weights.iloc[:,0:-1]
        war = temp_df.sum(axis=1).iloc[0]+midfielder_bias
        midfielder_df.loc[i,'WAR'] = war

    for i in range(len(defender_df)):

        temp_df = defender_df.iloc[i,:]*defender_weights.iloc[:,0:-1]
        war = temp_df.sum(axis=1).iloc[0]+defender_bias
        defender_df.loc[i,'WAR'] = war

    forward_df['WAR'] = forward_df['WAR']+(forward_df['WAR'].min()*-1)+1
    midfielder_df['WAR'] = midfielder_df['WAR']+(midfielder_df['WAR'].min()*-1)+1
    defender_df['WAR'] = defender_df['WAR']+(defender_df['WAR'].min()*-1)+1

    forward_df['WAR'] = round(forward_df['WAR']/forward_df['WAR'].median()*100,2)
    midfielder_df['WAR'] = round(midfielder_df['WAR']/midfielder_df['WAR'].median()*100,2)
    defender_df['WAR'] = round(defender_df['WAR']/defender_df['WAR'].median()*100,2)

    forward_df =forward_df.sort_values(by='WAR',ascending=False).reset_index(drop=True)
    midfielder_df =midfielder_df.sort_values(by='WAR',ascending=False).reset_index(drop=True)
    defender_df =defender_df.sort_values(by='WAR',ascending=False).reset_index(drop=True)

    csv_maker(forward_df,forward_save_path)
    csv_maker(midfielder_df,midfielder_save_path)
    csv_maker(defender_df,defender_save_path)
    

In [43]:
def fm_cal_war(target_path,weight_path,season):

    forward_weights = pd.read_csv(weight_path+'/'+season+'/'+'forward_weights.csv',encoding='utf-8')
    defender_weights = pd.read_csv(weight_path+'/'+season+'/'+'defender_weights.csv',encoding='utf-8')
    midfielder_weights = pd.read_csv(weight_path+'/'+season+'/'+'midfielder_weights.csv',encoding='utf-8')

    
    # 매 폴더를 돌며
    season_path = ''
    season_path = target_path+'/'+season
    file_list = os.listdir(season_path)

    os.makedirs(target_path+'/'+season+'/WAR',exist_ok=True)

    for file in file_list:
        
        file_path = season_path+'/'+file
        #print(file_path)
        war_calculator(file_path,season_path,forward_weights,defender_weights,midfielder_weights)

In [49]:
target_path = './datas/FM_DATA_MERGED'
weight_path = './datas/FM_DATA_WEIGHT'
season = '2020-2021'

fm_cal_war(target_path,weight_path,season)